## Libraries

In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))

In [2]:
import json
import numpy as np
from typing import List, Dict, Any
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from scripts.Tools import read_dialogue, get_conversation_by_id 

## Data

In [3]:
dataset = "Books"

In [4]:
with open(f"../data/{dataset}/item_map.json", "r") as f:
    item_map = json.load(f)

with open(f"../data/{dataset}/final_data.jsonl", "r") as f:
    final_data = [json.loads(line) for line in f]

dialogues = read_dialogue(f"../data/{dataset}/Conversation.txt")

In [5]:
items = list(set([item_map[key].lower().strip() for key in item_map]))

In [6]:
item_interactions = {}
for user_data in final_data:
    for user_id in user_data:
        for item_id in user_data[user_id]["history_interaction"]:
            item_name = item_map[item_id].lower().strip()
            if item_name not in item_interactions:
                item_interactions[item_name] = 0
            item_interactions[item_name] += 1

In [7]:
def get_messages(conversation_id: int, dialogues: str) -> List[BaseMessage]:

    conversation = get_conversation_by_id(dialogues, conversation_id)
    lines = conversation.strip().split("\n\n")
    messages = []

    for line in lines:
        if line.startswith("User:"):
            messages.append(HumanMessage(content=line[5:].strip()))
        elif line.startswith("Agent:"):
            messages.append(AIMessage(content=line[6:].strip()))
    
    return messages

## Models 1

In the following subsection, we implement three baseline models for conversational recommendation:

1. Random recommendation
2. Most popular recommendation
3. Zero-shot recommendation


In [8]:
def recommend_random(items: List[str], n: int = 5):
    return np.random.choice(items, n, replace=False).tolist()


def recommend_most_popular(item_interactions: Dict[str, int], n: int = 5):
    most_popular_items = sorted(
        item_interactions, key=item_interactions.get, reverse=True
    )[:n]
    return most_popular_items


def recommend_zero_shot(messages: List[BaseMessage], n: int = 5, verbose: bool = False):

    class Recommendation(BaseModel):
        items: Any = Field(
            description="A list of items. Each item must contain a 'reason' and a 'name'"
        )

    system = """
        You are a recommender system.
        Recommend {n} items based on the user's conversation.
        Sort the items by descending order of relevance.

        Conversation:
        {messages}
        """

    prompt = ChatPromptTemplate.from_messages([("system", system)])

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    structured_llm = llm.with_structured_output(Recommendation, include_raw=True)

    zero_shot_structured_llm = prompt | structured_llm
    response = zero_shot_structured_llm.invoke({"messages": messages, "n": n})

    prompt_tokens = response["raw"].response_metadata["token_usage"]["prompt_tokens"]
    completion_tokens = response["raw"].response_metadata["token_usage"][
        "completion_tokens"
    ]

    cost = 0.150 * prompt_tokens / 1000000 + 0.600 * completion_tokens / 1000000

    if verbose:
        print(f"Prompt Tokens: {prompt_tokens}")
        print(f"Completion Tokens: {completion_tokens}")
        print(f"Cost: ${cost} USD")

    return response["parsed"].items

In [9]:
recommend_random(items)

['justice for laine (badge of honor: texas heroes) (volume 4)',
 'nirv holy bible: the best translation for understanding gods word',
 'the edible woman',
 'forty signs of rain (science in the capital)',
 "flippin' the hustle (wahida clark presents)"]

In [10]:
recommend_most_popular(item_interactions)

['gone girl',
 'the girl on the train',
 'the pillars of the earth',
 'the old man and the sea',
 'books" />']

In [11]:
messages = get_messages(0, dialogues)
recommend_zero_shot(messages[:1], n=3, verbose=False)

[{'name': '"The Perfect Couple"',
  'reason': "This book features a gripping drama with complex relationships and unexpected twists, similar to the engaging storytelling in 'We Both Can't Be Bae (Volume 1)'."},
 {'name': '"The Seven Husbands of Evelyn Hugo"',
  'reason': 'A captivating tale of love, betrayal, and the glamorous yet tumultuous life of a Hollywood icon, which will keep you hooked just like your previous read.'},
 {'name': '"The Nightingale"',
  'reason': 'This historical fiction novel is rich in drama and emotional depth, focusing on the lives of two sisters during World War II, making it a compelling read for fans of intense narratives.'}]

## Models 2

In the following subsection, we implement the first agent.

### Vector database

The most important tool is the vector database, as it allows to interact semantically with the currrent catalog of books.

In [12]:
import tiktoken
from uuid import uuid4
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [13]:
def get_token_count(text: str) -> int:
    model = "text-embedding-3-small"
    encoding_name = tiktoken.encoding_for_model(model).name
    encoding = tiktoken.get_encoding(encoding_name)
    total_tokens = len(encoding.encode(text))

    return total_tokens

def get_embedding_cost(num_tokens: int) -> float:
    cost_per_million_tokens = 0.020
    return cost_per_million_tokens * (num_tokens / 1_000_000)

In [14]:
documents = []
for item in items:
    if item != "":
        documents.append(Document(page_content=f"Book: {item}"))

In [15]:
len(documents)

168823

In [16]:
total_tokens = 0
expected_cost = 0
for doc in documents:
    tokens = get_token_count(doc.page_content)
    total_tokens += tokens
    expected_cost += get_embedding_cost(tokens)
print(f"Total tokens: {total_tokens}")
print(f"Expected cost: {expected_cost}")

Total tokens: 2150836
Expected cost: 0.04301671999999588


In [17]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

store = Chroma(
    collection_name="books",
    embedding_function=embeddings,
    persist_directory="../store",
)

In [18]:
#uuids = [str(uuid4()) for _ in documents]
# Iterate in batches of 10,000
#for i in range(0, len(documents), 10_000):
#    print(f"Adding documents {i} to {i + 10_000}")
#    store.add_documents(documents=documents[i : i + 10_000], ids=uuids[i : i + 10_000])

### Agent

In [21]:
from typing import Annotated, Sequence, Optional
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.tools.retriever import create_retriever_tool
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables import RunnableLambda
from langchain_community.tools import TavilySearchResults

In [22]:
tavily_tool = TavilySearchResults(
    max_results=3,
    search_depth="advanced",
    include_answer=False,
    include_raw_content=False,
    include_images=False,
    description="Search the web for information about books.",
    )

books_retriever = create_retriever_tool(
    store.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    "retrieve_books",
    "Search and return books in the database.",
)

In [44]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    recommendations: Optional[Sequence[str]] = None


async def agent(state):

    messages = state["messages"]

    prompt = PromptTemplate.from_template(
        template="""
            You are a conversational recommender system specialized in books.
            Given a current conversation, recommend 3 books that align with the user's interests.

            Current conversation is as follows:
            {messages}
        """
    )

    llm = ChatOpenAI(
        temperature=0,
        streaming=True,
        model="gpt-4o-mini",
        model_kwargs={"stream_options": {"include_usage": True}},
    )

    tools = [books_retriever, tavily_tool]

    llm = llm.bind_tools(tools)

    chain = prompt | llm

    response = await chain.ainvoke({"messages": messages})

    return {"messages": [response]}


def should_continue(state):

    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return END
    else:
        return "tools"

In [45]:
workflow = StateGraph(AgentState)
tools = ToolNode([books_retriever, tavily_tool])

workflow.add_node("agent", RunnableLambda(agent))
workflow.add_node("tools", tools)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges("agent", should_continue, {END: END, "tools": "tools"})
workflow.add_edge("tools", "agent")

workflow.add_edge("agent", END)

graph = workflow.compile()

In [46]:
messages = get_messages(0, dialogues)

In [50]:
output = await graph.ainvoke({"messages": messages[0:1]})

In [53]:
output

{'messages': [HumanMessage(content='Hello! I recently read "We Both Can\'t Be Bae (Volume 1)" and I really enjoyed it. The drama in the story kept me hooked. Can you recommend another book similar to it?', additional_kwargs={}, response_metadata={}, id='ab245e5e-b480-4df9-b848-9e4ec39409fb'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Degr7uzx7HiTDYubrdfZoIkX', 'function': {'arguments': '{"query":"We Both Can\'t Be Bae (Volume 1)"}', 'name': 'retrieve_books'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1'}, id='run-56ebe55a-a501-4a9b-a8e1-2f76fa4502e4-0', tool_calls=[{'name': 'retrieve_books', 'args': {'query': "We Both Can't Be Bae (Volume 1)"}, 'id': 'call_Degr7uzx7HiTDYubrdfZoIkX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 203, 'output_tokens': 24, 'total_tokens': 227, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'outp

In [51]:
def get_inference_cost(messages: list[BaseMessage]) -> float:
    input_tokens = 0
    output_tokens = 0

    for message in messages:
        if isinstance(message, AIMessage):
            input_tokens += message.usage_metadata["input_tokens"]
            output_tokens += message.usage_metadata["output_tokens"]
    return input_tokens * 0.150 / 1_000_000 + output_tokens * 0.0600 / 1_000_000

In [54]:
get_inference_cost(output["messages"])

0.00091578